In [ ]:
!pip install memory-profiler openpyxl

In [ ]:
import json, os
for k,v in json.load(open("local.settings.json"))["Values"].items():
    os.environ[k] = v

In [ ]:
import pandas as pd
from libs.utils.esquire.neighbors.logic_vectorized import get_all_neighbors
from memory_profiler import memory_usage

# Load the data
df = pd.read_excel("real_sample_formatted.xlsx").drop_duplicates().head(100)

# Define a wrapper function to run the profiling
def run_get_all_neighbors():
    return get_all_neighbors(df, 3)

# Profile the memory usage
mem_usage = memory_usage(run_get_all_neighbors, interval=0.1)


In [ ]:
import matplotlib.pyplot as plt
plt.plot(mem_usage)
plt.xlabel('Time (s)')
plt.ylabel('Memory Usage (MiB)')
plt.title('Memory Usage Over Time')
plt.show()

# Or simply print the maximum memory usage
print(f"Maximum memory usage: {max(mem_usage)} MiB")